# Plots of baseline by Appointments and by Patients

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import os
from sqlalchemy import create_engine

import pandas as pd

In [ ]:
%matplotlib inline

from IPython.display import Image

In [ ]:
pd.options.display.max_columns = 200

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
dburl = os.environ['DBURL']
engine = create_engine(dburl)
connection = engine.connect()

----

In [ ]:
query = f"""select entity_id, outcome_date::date as outcome_date,
    outcome
    from events_on_appt_dates_cs
    where outcome_date is not null
    and extract(year from outcome_date) < 2016;"""
data_retention = pd.read_sql(query, connection)
data_retention['year'] = data_retention['outcome_date'].apply(lambda x: x.year)

query = f"""select entity_id, outcome_date::date as outcome_date,
    no_access_6mo::bool as outcome
    from access_ucm_6months
    where outcome_date is not null
    and extract(year from outcome_date) < 2016;"""
data_access = pd.read_sql(query, connection)
data_access['year'] = data_access['outcome_date'].apply(lambda x: x.year)

In [ ]:
def get_baseline_by_appt_patient(data):
    df1 = data.groupby('year')['entity_id'].nunique().to_frame().reset_index()
    df2 = data[data.outcome].groupby('year')['entity_id'].nunique().to_frame().reset_index()
    df_patient = df1.merge(df2, on='year')

    df_patient = df_patient.rename(index=str, columns={"entity_id_y": "count_ooc", "entity_id_x":"count"})
    df_patient.loc[:, "frac_out_of_care"] = df_patient["count_ooc"]/df_patient["count"]

    df1 = data.groupby('year')['entity_id'].count().to_frame().reset_index()
    df2 = data[data.outcome].groupby('year')['entity_id'].count().to_frame().reset_index()
    df_appt = df1.merge(df2, on='year')

    df_appt = df_appt.rename(index=str, columns={"entity_id_y": "count_ooc", "entity_id_x":"count"})
    df_appt.loc[:, "frac_out_of_care"] = df_appt["count_ooc"]/df_appt["count"]
    
    return(df_appt, df_patient)

In [ ]:
df_retention_appt, df_retention_patient = get_baseline_by_appt_patient(data_retention)
df_access_appt, df_access_patient = get_baseline_by_appt_patient(data_access)

In [ ]:
plt.close('all')
sns.set_style('whitegrid', {'axes.grid' : False})
sns.set_context("poster", font_scale=1, rc={"lines.linewidth": 3,"lines.markersize":12})
plt.rc('xtick', labelsize='small')
plt.rc('ytick', labelsize='small')

fig, ax = plt.subplots(1,figsize=(18, 8))

_ = ax.plot(df_retention_patient["year"], df_retention_patient["frac_out_of_care"], '--', marker='o', color='#374E55FF', label="Retention")
_ = ax.plot(df_access_patient["year"], df_access_patient["frac_out_of_care"], '--', marker='^', color='#DF8F44FF', label="Access in 6 months")

_ = plt.ylim(0,1)
_ = ax.set_xlabel('Year')
_ = ax.set_ylabel('Percentage of Patients Out of Care', color='black')
_ = ax.tick_params('y', colors='black')
_ = plt.legend(bbox_to_anchor=(0., 1, 1., -0.05), loc=9,ncol=2, borderaxespad=0.)
vals = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in vals])
ax.yaxis.tick_left()
ax.xaxis.tick_bottom()
fig.tight_layout()

In [ ]:
plt.close('all')
sns.set_style('whitegrid', {'axes.grid' : False})
sns.set_context("poster", font_scale=1, rc={"lines.linewidth": 3,"lines.markersize":12})
plt.rc('xtick', labelsize='small')
plt.rc('ytick', labelsize='small')

fig, ax = plt.subplots(1,figsize=(18, 8))

_ = ax.plot(df_retention_appt["year"], df_retention_appt["frac_out_of_care"], '-', marker='o', color='#374E55FF', label="Retention")
_ = ax.plot(df_access_appt["year"], df_access_appt["frac_out_of_care"], '-', marker='^', color='#DF8F44FF', label="Access in 6 months")

_ = plt.ylim(0,1)
_ = ax.set_xlabel('Year')
_ = ax.set_ylabel('Percentage of Appointments Out of Care', color='black')
_ = ax.tick_params('y', colors='black')
_ = plt.legend(bbox_to_anchor=(0., 1, 1., -0.05), loc=9,ncol=2, borderaxespad=0.)
vals = ax.get_yticks()
_ = ax.set_yticklabels(['{:.0%}'.format(x) for x in vals])
ax.yaxis.tick_left()
ax.xaxis.tick_bottom()
fig.tight_layout()